In [1]:
import sys
sys.path.append("..")

import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np
import os
from os.path import exists

import pymc as pm
import pytensor
import pytensor.tensor as pt
import matplotlib.pyplot as plt
import seaborn as sns
import arviz as az

config = {
    'figure.figsize':(14,4),
    'figure.constrained_layout.use':True,
    'figure.facecolor':'w',
    'axes.grid':True,
    'grid.linewidth':0.5,
    'grid.linestyle':'--',
    'axes.spines.top':False,
    'axes.spines.bottom':False,
    'axes.spines.left':False,
    'axes.spines.right':False
}

plt.rcParams.update(config)

SEED = sum(list(map(ord, 'climate_bayes')))
rng = np.random.default_rng(SEED)

In [2]:
from laos_gggi.combine_data import final_data

# Data load

In [3]:
#Load data
data = final_data()
df_clim = data["df_time_series"][["co2", "Temp", "precip"]].iloc[1:-1] 
emdat_damage = data["emdat_damage"]["Total_Damage_Adjusted"]
development_indicators = data["wb_data"]

#Fill NaN values
emdat_damage_filled = emdat_damage.fillna(0)
